In [255]:
import pandas as pd
import numpy as np
import math
import operator

In [256]:
data1 = pd.read_csv("./MainTable_WatWin.csv")
data2 = pd.read_csv("./ErrorType.csv")

In [257]:
data1.rename(columns={'ErrType':'CompileErrorType'}, inplace=True)
data1.replace('N/A',np.nan)
data2.rename(columns={'Code':'CompileErrorMessage'}, inplace=True)
data2["CodeStateSection"] = ["hw_" + str(i) + ".py" if data2["IsErr"].iloc[i-1] == "Yes" else np.nan for i in data2["Unnamed: 0"]]
data1["SourceLocation"] = ["Text:" + str(data1["ErrLine"].iloc[i]) if data1["IsErr"].iloc[i] == "Yes" else np.nan for i in data1["Unnamed: 0"]] 
data1["CompileErrorMessage"] = [data2["CompileErrorMessage"].iloc[int(data1["CodeStateID"].iloc[i])-1] if data1["IsErr"].iloc[i] == "Yes" else np.nan for i in data1["Unnamed: 0"]]
data1["CodeStateSection"] = [data2["CodeStateSection"].iloc[int(data1["CodeStateID"].iloc[i])-1] if data1["IsErr"].iloc[i] == "Yes" else np.nan for i in data1["Unnamed: 0"]]

In [258]:
result_df1 = data1[['EventType', 'EventID', 'Order', 'SubjectID', 'ToolInstances', 'CodeStateID', 'ServerTimestamp', 'ProblemID', 'Correct', 'CompileErrorType', 'CompileErrorMessage', 'CodeStateSection', 'SourceLocation']].copy()

In [259]:
add_df1 = result_df1.copy()
add_df1 = add_df1[add_df1["EventType"]=="Submit"]
add_df1 = add_df1.replace("Submit", "Compile")
add_df1["Order"] += .1
add_df1["Correct"] = np.nan
add_df1["CompileErrorType"] = np.nan
add_df1["CompileErrorMessage"] = np.nan
add_df1["CodeStateSection"] = np.nan
add_df1["SourceLocation"] = np.nan
add_df1["ParentEvent"] = np.nan

In [260]:
add_df2 = result_df1.copy()
add_df2 = add_df2[add_df2["EventType"]=="Submit"]
add_df2 = add_df2[pd.isna(add_df2["CompileErrorType"])==False]
add_df2 = add_df2.replace("Submit", "Compile.Error")
add_df2["Order"] += .2
add_df2["Correct"] = np.nan
add_df2["ParentEvent"] = add_df2["Order"]-.1

In [261]:
result_df1["CompileErrorType"] = np.nan
result_df1["CompileErrorMessage"] = np.nan
result_df1["CodeStateSection"] = np.nan
result_df1["SourceLocation"] = np.nan
result_df1["ParentEvent"] = np.nan

In [262]:
result_df1 = result_df1.append(add_df1)
result_df1 = result_df1.append(add_df2)

In [263]:
result_df1 = result_df1.sort_values(['SubjectID', 'Order'], ascending=[True, True])

In [267]:
result_df1.to_csv("./ProgSnap2Tables/MainTable.csv", index=False)

In [266]:
result_df1

,EventType,EventID,Order,SubjectID,ToolInstances,CodeStateID,ServerTimestamp,ProblemID,Correct,CompileErrorType,CompileErrorMessage,CodeStateSection,SourceLocation,ParentEvent
0,Submit,f48cba2510087c502dde920d948b364b,1373.0,S002,ITAP; Python,1061,2016-02-05T19:46:00,nearestBusStop,True,NaN,NaN,NaN,NaN,NaN
0,Compile,f48cba2510087c502dde920d948b364b,1373.1,S002,ITAP; Python,1061,2016-02-05T19:46:00,nearestBusStop,NaN,NaN,NaN,NaN,NaN,NaN
1,Submit,93232279e317e6a46f6bdf001ab13a39,1374.0,S002,ITAP; Python,1062,2016-02-05T19:48:00,backwardsCombine,True,NaN,NaN,NaN,NaN,NaN
1,Compile,93232279e317e6a46f6bdf001ab13a39,1374.1,S002,ITAP; Python,1062,2016-02-05T19:48:00,backwardsCombine,NaN,NaN,NaN,NaN,NaN,NaN
2,X-HintRequest,4468830a73173d348adbb2159ee9a0fa,1375.0,S002,ITAP; Python,1063,2016-02-05T19:49:00,firstAndLast,False,NaN,NaN,NaN,NaN,NaN
3,X-HintRequest,2961b2aedf2483e7cf584a43aca4192e,1376.0,S002,ITAP; Python,1064,2016-02-05T19:50:00,firstAndLast,False,NaN,NaN,NaN,NaN,NaN
4,X-HintRequest,0f36ef933aaf49a51243ec78543a995d,1377.0,S002,ITAP; Python,1065,2016-02-05T19:51:00,firstAndLast,False,NaN,NaN,NaN,NaN,NaN
5,X-HintRequest,5c70d95383a9140d5ff6ed5b1c164032,1378.0,S002,ITAP; Python,1066,2016-02-05T19:51:00,firstAndLast,False,NaN,NaN,NaN,NaN,NaN
6,X-HintRequest,4b7b63d2a540bc28aa9f1a7efa8437aa,1379.0,S002,ITAP; Python,1066,2016-02-05T19:52:00,firstAndLast,False,NaN,NaN,NaN,NaN,NaN
7,X-HintRequest,938c0a3c1517c59373c550ab59707a09,1380.0,S002,ITAP; Python,1067,2016-02-05T19:52:00,firstAndLast,False,NaN,NaN,NaN,NaN,NaN


In [85]:
data1

,Unnamed: 0,EventType,EventID,Order,SubjectID,ToolInstances,CodeStateID,ServerTimestamp,ProblemID,Correct,IsErr,CompileErrorType,ErrLine,TimeEst,TimeMean,TimeStd,WatWin,SourceLocation,CompileErrorMessage,CodeStateSection
0,0,Submit,f48cba2510087c502dde920d948b364b,1373,S002,ITAP; Python,1061,2016-02-05T19:46:00,nearestBusStop,True,No,NaN,NaN,-1,52,64.284869,0.545813,NaN,NaN,NaN
1,1,Submit,93232279e317e6a46f6bdf001ab13a39,1374,S002,ITAP; Python,1062,2016-02-05T19:48:00,backwardsCombine,True,No,NaN,NaN,-1,52,64.284869,0.545813,NaN,NaN,NaN
2,2,X-HintRequest,4468830a73173d348adbb2159ee9a0fa,1375,S002,ITAP; Python,1063,2016-02-05T19:49:00,firstAndLast,False,Yes,Indentation,4,-1,52,64.284869,0.545813,Text:4,"File ""hw_1063.py"", line 4\n return\n ^...",hw_1063.py
3,3,X-HintRequest,2961b2aedf2483e7cf584a43aca4192e,1376,S002,ITAP; Python,1064,2016-02-05T19:50:00,firstAndLast,False,No,NaN,NaN,-1,52,64.284869,0.545813,NaN,NaN,NaN
4,4,X-HintRequest,0f36ef933aaf49a51243ec78543a995d,1377,S002,ITAP; Python,1065,2016-02-05T19:51:00,firstAndLast,False,Yes,Indentation,3,-1,52,64.284869,0.545813,Text:3,"File ""hw_1065.py"", line 3\n return\n ^...",hw_1065.py
5,5,X-HintRequest,5c70d95383a9140d5ff6ed5b1c164032,1378,S002,ITAP; Python,1066,2016-02-05T19:51:00,firstAndLast,False,No,NaN,NaN,-1,52,64.284869,0.545813,NaN,NaN,NaN
6,6,X-HintRequest,4b7b63d2a540bc28aa9f1a7efa8437aa,1379,S002,ITAP; Python,1066,2016-02-05T19:52:00,firstAndLast,False,No,NaN,NaN,-1,52,64.284869,0.545813,NaN,NaN,NaN
7,7,X-HintRequest,938c0a3c1517c59373c550ab59707a09,1380,S002,ITAP; Python,1067,2016-02-05T19:52:00,firstAndLast,False,No,NaN,NaN,-1,52,64.284869,0.545813,NaN,NaN,NaN
8,8,Submit,21ec3c4beaf552084cad7b60350767a9,1381,S002,ITAP; Python,1068,2016-02-05T19:53:00,firstAndLast,False,Yes,Syntax,2,60,52,64.284869,0.545813,Text:2,"File ""hw_1068.py"", line 2\n return + '[e...",hw_1068.py
9,9,Submit,d5f06ee0757f7c63be6aa1d9962b7df0,1382,S002,ITAP; Python,1069,2016-02-05T19:54:00,firstAndLast,False,Yes,Indentation,3,120,52,64.284869,0.545813,Text:3,"File ""hw_1069.py"", line 3\n return a\n ...",hw_1069.py


In [45]:
data2

,Unnamed: 0,CompileErrorMessage,IsErr,ErrLoc,ErrType,ErrLineIdx,ErrLineLoc,ErrLine,CodeStateSection
0,1,"File ""hw_1.py"", line 2\n for num <= n:\n ...",Yes,64,Syntax,18,18,2,hw_1.py
1,2,"File ""hw_2.py"", line 2\n for num <= in n:...",Yes,67,Syntax,18,18,2,hw_2.py
2,3,hello world!\n,No,NaN,NaN,NaN,NaN,NaN,NaN
3,4,hello world!\n,No,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
5,6,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
6,7,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Hello World!\n,No,NaN,NaN,NaN,NaN,NaN,NaN
8,9,"File ""hw_9.py"", line 2\n for num <= in ra...",Yes,74,Syntax,18,18,2,hw_9.py
9,10,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
